In [ ]:
#| default_exp scripts/update_assistant_files

In [ ]:
#| export
import argparse
import sys
from typing import List
import openai
import pandas as pd

/tmp/ipykernel_7212/2795621024.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#| export
def _set_assistant_files(assistant_id: str, file_ids: List[str], openai_client: openai.OpenAI) -> None:
    remove_files = set()
    existing_files = set()
    for existing_file in openai_client.beta.assistants.files.list(assistant_id=assistant_id).data:
        if existing_file.id not in file_ids:
            remove_files.add(existing_file.id)
        else:
            existing_files.add(existing_file.id)
    new_files = set(file_ids) - existing_files
    for file_id in remove_files:
        openai_client.beta.assistants.files.delete(
            file_id=file_id,
            assistant_id=assistant_id
        )
    for file_id in new_files:
        openai_client.beta.assistants.files.create(
            file_id=file_id,
            assistant_id=assistant_id
        )
    return file_ids

In [ ]:
#| export
def process(file_name_assistants: str, file_name_file_ids: str, file_name_openai_api_key: str, file_name_mapping: str) -> None:
    df_assistants = pd.read_csv(file_name_assistants)
    df_files = pd.read_json(file_name_file_ids, orient="records", lines=True)
    with open(file_name_openai_api_key, "r", encoding="utf-8") as src:
        openai_api_key = src.read()
    openai_client = openai.OpenAI(api_key=openai_api_key)
    df_assistants["file_ids"] = df_assistants["assistant_id"] \
        .apply(lambda assistant_id: _set_assistant_files(assistant_id, set(df_files["file_id"]), openai_client)) \
        .apply(set).apply(sorted).apply(",".join)
    df_assistants[["assistant_id", "file_ids"]].to_json(file_name_mapping, orient="records", lines=True)

In [ ]:
process(
    "../data/assistants.csv",
    "../data/openai-file-ids.jsonl",
    "../data/openaiapikey.txt",
    "../data/assistants-files-mapping.jsonl"
)

In [ ]:
# | export
if __name__ == "__main__" and "ipykernel_launcher" not in " ".join(sys.argv):
    parser = argparse.ArgumentParser()
    parser.add_argument("--file_name_assistants",
                        type=str,
                        required=True,
                        help="A CSV with assistant ids")
    parser.add_argument("--file_name_file_ids",
                        type=str,
                        required=True,
                        help="JSONL file with all the file ids")
    parser.add_argument("--file_name_openai_api_key",
                        type=str,
                        required=True,
                        help="OpenAI api key")
    parser.add_argument("--file_name_mapping",
                        type=str,
                        required=True,
                        help="Output assistant-file ids mapping")
    process(**vars(parser.parse_args()))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()